# For Day 1 recommendation

In [1]:
import gzip
import copy
import csv
import json
import random
import glob
import time
from datetime import date
from datetime import timedelta
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
import urllib
from urllib.error import HTTPError
import numpy as np
from numpy.linalg import norm
import pandas as pd
import scipy
import scipy.optimize
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from string import Template
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import codecs
from mailchimp_marketing import Client
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

In [2]:
global api_key
api_key = 'BBaiiKLwZjQGXNdWSNYKQ5mBonakgLrR' # set your api_key in https://newsapi.org/register
global mail_api_key
mail_api_key = '************************'
global mail_server
mail_server = 'us14'
wv = api.load('word2vec-google-news-300')

In [3]:
def save_dic_to_json(data, path):
    json_data = json.dumps(data)
    json_file = open(path, 'w')
    json_file.write(json_data)
    json_file.close()
    return True

In [ ]:
# Initial Utility Matrix using MIND dataset and decompose to get Item-matrix and User-matrix

In [4]:
# function part

def load_behavior_dataset(pretrain_data):
    # load file
    with open(pretrain_data) as file:
        dataset = []
        for line in file:
            fields = line.strip().split('\t')
            header = ['impression_id','user_id','time','history','impressions']
            d = dict(zip(header, fields))
            dataset.append(d)

    dataset_copy = dataset.copy()
    # only need subset of data
    dataset_sub = dataset_copy[:1000]

    # modify value of key history and inpression to be list structure
    for i in range(len(dataset_sub)):
        dataset_sub[i]['history'] = dataset_sub[i]['history'].split()
        dataset_sub[i]['impressions'] = dataset_sub[i]['impressions'].split()

    for i in range(len(dataset_sub)):
        dataset_sub[i]['history'] = [str(x) + '-1' for x in (dataset_sub[i]['history'])]

    # combine history and impression to one key as rate
    # then convert rate value from 'news-click' to {news:click}
    for i in range(len(dataset_sub)):
        dataset_sub[i]['click'] = dataset_sub[i]['history'] + dataset_sub[i]['impressions']
        dataset_sub[i]['click'] = list(s.split('-',1) for s in dataset_sub[i]['click'])
        dataset_sub[i].pop('history', None)
        dataset_sub[i].pop('impressions', None)

    output = []
    for i in range(len(dataset_sub)):
        for u_click in dataset_sub[i]['click']:
            output.append({'user_id': dataset_sub[i]['user_id'], 'time': dataset_sub[i]['time'], 'news_id': u_click[0],'click':int(u_click[1])} )

    # save data into dataframe named 'data'
    data = pd.DataFrame([output[0]], columns=output[0].keys())
    for i in range(1,len(output)):
        data2 = pd.DataFrame([output[i]], columns=output[i].keys())
        data = pd.concat([data,data2],ignore_index=True)
        
    data['click'] = data['click']+1
    return data


def load_news_dataset(path):
    with open(path) as file:
        dataset = []
        for line in file:
            fields = line.strip().split('\t')
            header = ['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
            d = dict(zip(header, fields))
            dataset.append(d)
    return dataset

def get_news_text(url):
    try:
        response = urllib.request.urlopen(url)
    except HTTPError:
        return ''
    else:
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, features="lxml")
        doc = []
        content = ''
        p_list = soup.find_all('p')
        p_list_new = []
        for i in p_list:
            for text in i.strings:
                doc.append(text)
        content = ''.join(doc)
        return content
    
def create_utility_matrix(df):

    user_list = df['user_id'].unique()
    news_list = df['news_id'].unique()

    # create utility natrix, all avlues nan
    utility_matrix = np.empty(shape = (len(user_list),len(news_list))) # create an empty 2d np array

    # fill in the value
    for i in range(len(df.index)):
        utility_matrix[np.where(user_list==df.iloc[i]['user_id'])[0].item()][np.where(news_list==df.iloc[i]['news_id'])[0].item()] = df.iloc[i]['click']

    return utility_matrix,user_list,news_list

def train_test_split(ut_matrix): # used for train hyperparameter k
    
    validation = np.zeros(ut_matrix.shape)
    train = ut_matrix.copy() #don't do train=ratings, other wise, ratings becomes empty
    
    for user in np.arange(ut_matrix.shape[0]):
        if len(ut_matrix[user,:].nonzero()[0])>=35:# 35 seems to be best, it depends on sparsity of your user-item matrix
            random.seed(0)
            val_click = np.random.choice(ut_matrix[user, :].nonzero()[0], 
                                        size=15, #tweak this, 15 seems to be optimal
                                        replace=False)
            train[user, val_click] = 0
            validation[user, val_click] = ut_matrix[user, val_click]
    print(validation.shape)
    return train, validation


def error(R,P,Q,lamda=0.02):
    clicks = R.data
    rows = R.row
    cols = R.col
    e = 0 
    for ui in range(len(clicks)):
        rui=clicks[ui]
        u = rows[ui]
        i = cols[ui]
        if rui>0:
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)+\
                lamda*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2))
    return e

def SGD(R, K, lamda=0.02,steps=100, gamma=0.001):
    
    M,N = R.shape
    random.seed(0)
    P = np.random.rand(M,K)
    Q = np.random.rand(K,N)
    
    rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
#     print("Initial RMSE: "+str(rmse))
    
    for step in range(steps):
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui=rui-np.dot(P[u,:],Q[:,i])
                P[u,:]=P[u,:]+gamma*2*(eui*Q[:,i]-lamda*P[u,:])
                Q[:,i]=Q[:,i]+gamma*2*(eui*P[u,:]-lamda*Q[:,i])
        rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
        if rmse<0.5:
            break
#     print("Final RMSE: "+str(rmse))
    return P,Q

def latent_process(utilMat):
    train, val = train_test_split(utilMat)
    R = coo_matrix(train)
    S = coo_matrix(val)
    UM = coo_matrix(utilMat)
    rmse_val_list = []
    k = [20,25,30,35,40,45,50]
    for ki in k:
        P,Q = SGD(R,ki)
        rmse = np.sqrt(error(S,P,Q)/len(S.data))
        rmse_val_list.append(rmse)
    index_min = min(range(len(rmse_val_list)), key=rmse_val_list.__getitem__)
    k_select = k[index_min]

    P,Q=SGD(UM,k_select)
    rmse = np.sqrt(error(UM,P,Q)/len(UM.data))
    print(f'rmse'+str(rmse))
    
    return P,Q

In [ ]:
# processing part

# load behavior dataset
behavior_data = load_behavior_dataset("../final_project/MINDlarge_train/behaviors.tsv")

# base on behavior dataframe, we can get the limited news' information and delete the news without text
news_dataset = load_news_dataset("../final_project/MINDlarge_train/news.tsv")
news_id_list = list(behavior_data.loc[:,'news_id'].unique()) # get news id list in behavior info board

news_dataset_sub = [] # select all news in behavior info board
for news in news_dataset:
    if news['News ID'] in news_id_list:
        news_dataset_sub.append(news)
        
for news in news_dataset_sub: # get full text information of news
    url = news['URL']
    text = get_news_text(url)
    news['text'] = text
    
# delete news without text
news_pd = pd.DataFrame.from_dict(news_dataset_sub)
index_list = news_pd[news_pd['text'] == ''].index
news_pd.drop(index_list, inplace = True)
news_pd.reset_index(drop=True, inplace=True)

news_dataset_sub = news_pd.to_dict('records')

# save raw news info
json_data = json.dumps(news_dataset_sub)
json_file = open('dataset/base_news/raw_news/base_news.json', 'w')
json_file.write(json_data)
json_file.close()

# form a id:title dict for futher processing
base_title_dict = {}
for i in range(len(news_pd)):
    news_id = news_pd.loc[i,'News ID']
    text = news_pd.loc[i,'Title'].lower()
    base_title_dict[news_id] = text

# save id-title info
json_data = json.dumps(news_dataset_sub)
json_file = open('dataset/base_news/title_news/news_id_title.json', 'w')
json_file.write(json_data)
json_file.close()

In [5]:
f = open('dataset/base_news/title_news/news_id_title.json', 'r', encoding='utf-8')
base_title_dict = json.load(f)

In [6]:
behavior_data = load_behavior_dataset("../final_project/MINDlarge_train/behaviors.tsv")

In [7]:
aval_news_id = list(base_title_dict.keys())
behavior_data = behavior_data[behavior_data['news_id'].isin(aval_news_id)]

utilMat, users_index, items_index = create_utility_matrix(behavior_data) # utilMat stores the utility matrix

sparsity = float(len(utilMat.nonzero()[0]))
sparsity /= (utilMat.shape[0] * utilMat.shape[1])
sparsity *= 100
print ('percentage of user-items that have a rating: {:.2f}%'.format(sparsity))

User_matrix, Item_matrix = latent_process(utilMat)

# save matrix
Item_matrix = pd.DataFrame(Item_matrix, columns = items_index)
Item_matrix.to_csv('dataset/P_Q_matrix/Item_matrix.csv', index=False)

User_matrix = pd.DataFrame(User_matrix, index=users_index)
User_matrix.to_csv('dataset/P_Q_matrix/User_matrix.csv')

Util_matrix = pd.DataFrame(utilMat,columns=items_index,index=users_index)
Util_matrix.to_csv('dataset/P_Q_matrix/Util_matrix.csv')

percentage of user-items that have a rating: 0.57%
(1000, 11917)
rmse0.49851121192548753


In [8]:
Util_matrix

,N8668,N39081,N65259,N79529,N73408,N43615,N29379,N32031,N110232,N101921,...,N57930,N122020,N91556,N92487,N84669,N48230,N95905,N101084,N105379,N22752
U87243,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U598644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U532401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U593596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U239687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U348354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U110667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U190481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U463199,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [9]:
Item_matrix

,N8668,N39081,N65259,N79529,N73408,N43615,N29379,N32031,N110232,N101921,...,N57930,N122020,N91556,N92487,N84669,N48230,N95905,N101084,N105379,N22752
0,0.877603,0.906361,0.778421,0.660243,0.431438,0.901328,0.171397,0.635892,0.192597,0.399430,...,0.049762,0.914065,0.417210,0.421932,0.474703,0.627201,0.074070,0.523428,0.207952,0.192038
1,0.456578,0.158354,0.226215,0.326932,0.936573,0.640263,0.887155,0.145461,0.745720,0.491712,...,0.375642,0.225424,0.773118,0.802582,0.027852,0.481925,0.214900,0.485022,0.285966,0.283665
2,0.414422,0.863837,0.781377,0.077351,0.700641,0.189755,0.217052,0.899357,0.501649,0.953466,...,0.746619,0.917710,0.967791,0.182862,0.460700,0.942878,0.653183,0.846826,0.319804,0.318497
3,0.715153,0.100578,0.544292,0.796145,0.261864,0.747882,0.271807,0.521569,0.018961,0.412325,...,0.110305,0.014271,0.685533,0.775083,0.296221,0.464271,0.688241,0.586132,0.109362,0.015452
4,0.285251,0.792175,0.252691,0.665473,0.480680,0.800329,0.081549,0.750682,0.437289,0.794363,...,0.554395,0.361541,0.347119,0.414938,0.417197,0.006260,0.948201,0.116899,0.643254,0.324658
5,0.812057,0.743292,0.137113,0.484495,0.045882,0.459632,0.452358,0.043151,0.028110,0.260201,...,0.267861,0.756692,0.364692,0.632023,0.362679,0.200912,0.623000,0.644320,0.130599,0.166685
6,0.623802,0.831238,0.111092,0.129999,0.852058,0.134528,0.207155,0.548561,0.207407,0.420608,...,0.149698,0.463750,0.967042,0.376804,0.362193,0.890857,0.507061,0.634342,0.278271,0.771390
7,0.616894,0.772998,0.922932,0.420197,0.157206,0.325468,0.868603,0.272919,0.361987,0.406080,...,0.201443,0.772003,0.123974,0.510541,0.621624,0.149830,0.635863,0.637306,0.447599,0.354474
8,0.767829,0.928352,0.100242,0.748499,0.121855,0.857677,0.392399,0.054427,0.356801,0.427514,...,0.440767,0.964643,0.151586,0.095539,0.534174,0.958366,0.182005,0.892799,0.806130,0.302244
9,0.145879,0.197632,0.693388,0.688840,1.002301,0.600832,0.480214,0.856809,0.657978,0.100345,...,0.979158,0.381715,0.734476,0.512113,0.292627,0.597863,0.527058,0.940642,0.284164,0.391430


In [10]:
User_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
U87243,-0.170834,0.385050,0.402222,0.073740,0.130955,-0.065805,0.220294,0.243431,0.183824,0.030802,0.444983,-0.056752,0.080407,0.036629,0.615524,-0.132659,0.624842,0.051917,0.591545,0.626010
U598644,0.270844,0.239972,0.016771,0.341753,-0.041445,0.085981,-0.107320,0.496710,-0.059350,-0.268793,0.360562,0.531367,0.322047,0.221719,-0.313538,0.311841,0.279891,0.395090,-0.031228,0.027580
U532401,0.760817,0.608568,0.706947,0.391984,0.850360,0.372120,-0.028698,0.060095,-0.140115,0.386638,-0.067479,0.624599,0.026453,0.121464,-0.005057,0.243132,0.063576,0.048203,0.116294,-0.041098
U593596,-0.118622,0.038558,-0.121559,0.090061,0.505711,0.212203,-0.027015,-0.081392,0.012020,0.219614,0.131692,0.410386,0.557017,0.143364,0.284889,0.367204,0.284677,0.004553,0.138729,0.304013
U239687,0.100024,0.290726,0.086042,0.165464,0.157505,0.083564,0.155413,0.146133,0.240869,0.150688,0.204806,0.151234,0.269307,0.048571,0.142383,0.075731,0.139981,0.207633,0.218089,0.191249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U348354,-0.048878,0.251354,-0.266399,0.618803,0.510920,0.175894,0.172831,0.147570,0.137899,-0.145501,-0.051228,0.416075,0.415006,0.110111,0.566035,0.025937,0.580417,0.232202,0.375888,0.045547
U110667,0.233970,0.082049,0.121637,0.055585,0.301337,-0.034569,0.376283,0.192396,0.208683,0.239402,0.131489,0.457430,0.137531,0.238527,-0.202631,0.444265,0.458158,0.405085,0.031582,0.151432
U190481,0.442329,0.377161,0.325374,0.612335,0.473616,0.279950,0.052198,0.530943,0.135452,-0.185793,0.258386,-0.055709,0.456491,-0.114208,0.634866,0.179858,0.463170,0.385318,0.262971,0.497384
U463199,-0.093771,0.266343,0.370258,0.212802,0.273558,0.106913,0.096379,0.153108,0.091324,0.027110,-0.122135,0.067611,-0.005472,0.311505,0.188974,-0.157180,0.204020,0.336737,0.412574,0.048002


In [12]:
P,Q = User_matrix.to_numpy(),Item_matrix.to_numpy()
SGD_prediction=np.dot(P,Q)
train, val = train_test_split(utilMat)
estimation= SGD_prediction[val.nonzero()]
ground_truth = val[val.nonzero()]
results=pd.DataFrame({'prediction':estimation, 'actual rating':ground_truth})
results

(1000, 11917)


,prediction,actual rating
0,1.540890,2.0
1,2.089042,2.0
2,2.324602,2.0
3,2.128768,2.0
4,1.895231,2.0
...,...,...
9775,1.108754,1.0
9776,0.875747,1.0
9777,1.077894,1.0
9778,0.969469,1.0


In [ ]:
# Crawl news

In [13]:
# function

def news_get_request(url):
    results = requests.get(url+api_key)
    json_data = results.json()
    return json_data

def get_recommand_news(year, month):
    start = time.time()
    request_url = 'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key='.format(year = year, month = month)
    json_data = news_get_request(request_url)
    news_json = {'data' : []}
    raw_data = json_data['response']['docs']
    today_date = date.today().strftime("%Y-%m-%d")
    for news in raw_data:
        data = {}
        # get publish date
        raw_date = news['pub_date']
        timestamp = raw_date.split('T')
        if timestamp[0] != today_date:
            continue
        # get author
        author = news['byline']['person']
        if author == []:
            continue
        else:
            name_comp = list(author[0].keys())[:3]
            author_list = []
            for person in author:
                name_list = []
                for comp in name_comp:
                    if person[comp] != None:
                        name_list.append(person[comp])
                name = ' '.join(name_list)
                author_list.append(name)
        # get caregory
        section = news['section_name']
        # get abstract
        abstract = news['abstract'] 
        # get url
        url = news['web_url']
        # get tital
        title = news['headline']['main']
        # get keywords
        kw_lsit = []
        for kw in news['keywords']:
            kw_lsit.append(kw['value'])
        # get id
        raw_id = news['_id']
        real_id = raw_id.split('/')[-1]
        # save info to dic
        data['id'] = real_id
        data['publish_data'] = timestamp[0]
        data['author'] = author_list
        data['url'] = url
        data['section'] = section
        data['title'] = title
        data['abstract'] = abstract
        data['keywords'] = kw_lsit
        news_json['data'].append(data)
    end = time.time()
    print('total # of news in dic：{}. total time: {}'.format(len(news_json['data']), end - start))
    return news_json

def get_dic_text(data): # collect urls that cannot open
    error_list = []
    total = len(data['data'])
    for i in range(total):
        start = time.time()
        temp = data['data'][i]
        url = temp['url']
        real_id = temp['id']
        text = get_news_text(url)
        if text == '':
            error_list.append(i)
        else:
            temp['text'] = text
        end = time.time()
        print('completing------{}/{}, using time: {}'.format(i+1, total, end - start))
    return error_list

def clean_empty_text(data):
    data_raw = data['data']
    data_new = data_raw[:]
    count = 0
    for i in range(len(data_raw)):
        cur = data_raw[i]
        if 'text' not in cur.keys():
            data_new.pop(i-count)
            count += 1
    return data_new

In [ ]:
# crawl today's news
today = date.today()
year = int(today.strftime("%Y"))
month = int(today.strftime("%m")) # year and month can be modified
day = int(today.strftime("%d"))
raw_news_json = get_initial_news(year, month)

# get text, if failed, add to error list, if failed too much, error list can use to re-get text
error_list = get_dic_text(raw_news_json)

# delete news without text
raw_news_json_copy = copy.deepcopy(raw_news_json)
news_json = clean_empty_text(raw_news_json_copy)
rec_title_list = {}
for news in raw_news_json['data']:
    n_id = news['id']
    title = news['title']
    rec_title_list[n_id] = title
    
# save data
save_dic_to_json(error_list, 'dataset/rec_news/error_list/{}.json'.format(today.strftime("%Y-%m-%d")))
save_dic_to_json(news_json, 'dataset/rec_news/raw_news/{}.json'.format(today.strftime("%Y-%m-%d")))
save_dic_to_json(rec_title_list, 'dataset/rec_news/title_news/{}.json'.format(today.strftime("%Y-%m-%d")))

In [14]:
today = date(2022, 2, 1)

In [ ]:
# use w2v transfer title

In [15]:
# function

def tokenize(text):
    punc = '''!()[]{};:'"\,<>.?/@#—$%―^&*_~–-+=|…“”’‘`”'''
    number = '''1234567890'''
    text = text.lower()
    for i in text: # remove punctuation and lowercase text
        if i in punc or i in number:
            text = text.replace(i, ' ')
    words = text.split()
    return words

def processing_corpus(data, model):
    index = list(data.keys())
    corpus = list(data.values())
    sent2vec_dic = {}
    for i in range(len(corpus)):
        tokens = tokenize(corpus[i])
        count = 0
        sent_vec = np.zeros(model.vector_size)
        for token in tokens:
            try:
                token_vec = model[token]
            except KeyError:
                continue
            count += 1
            sent_vec += token_vec
        if count != 0:
            sent_vec = sent_vec/count
        else:
            continue
        sent2vec_dic[index[i]] = list(sent_vec)
    return sent2vec_dic

In [16]:
f = open('dataset/rec_news/title_news/2022-02-01.json', 'r', encoding='utf-8')
rec_title_dict = json.load(f)

In [17]:
# transfer base news {id:title} to {id:vector}
base_title_vec = processing_corpus(base_title_dict, wv)

# transfer recommend news {id:title} to {id:vector}
rec_title_vec = processing_corpus(rec_title_dict, wv)

In [ ]:
# calculate similarity, get recommand news latent matrix

In [18]:
# function

def cal_rec_matrix(base_title_vec_pd, rec_title_vec_pd, Item_matrix):
    rec_news_matrix = pd.DataFrame()
    q_list = []
    for i in range(len(rec_news_pd)):
        matrix = cosine_similarity(base_news_pd, [rec_news_pd.iloc[i, :]])
        test = Item_matrix.dot(matrix/sum(matrix))
        q_list.append(test)
    rec_news_matrix = pd.concat(q_list, axis=1)
    rec_news_matrix.columns = rec_news_pd.index
    return rec_news_matrix

In [19]:
base_news_pd = pd.DataFrame.from_dict(base_title_vec, orient='index')
rec_news_pd = pd.DataFrame.from_dict(rec_title_vec, orient='index')
rec_news_matrix = cal_rec_matrix(base_news_pd, rec_news_pd, Item_matrix) # new item matrix: row:latent factor; column: news id

In [20]:
Item_matrix_new = pd.concat([Item_matrix, rec_news_matrix], axis=1)
Item_matrix_new.to_csv('dataset/P_Q_matrix/Item_matrix_u1.csv',index=False)

In [21]:
rec_news_matrix

,fe93a16a-33c1-5cee-a16d-0c957186afe7,112e1047-66da-52ac-81dd-02265758564b,2dde2ecf-7352-5ca6-be5b-c0b8dd19480c,1f993990-0666-5f54-9de8-957abfcb93d7,ae94f6de-6207-55bc-bf5f-fddc81259cad,0e053b35-9c26-54d4-9219-d697ad1cfd69,a5a30a29-943e-5ef4-84fc-158000411e0f,8d69e5f6-7a3c-5a72-8dd9-41705c961b6a,5ff1073b-0f8b-53a4-9049-0a7fa76509de,f938cdf7-cdd1-57d4-b77e-7347b6848481,...,225e2df1-bfc6-5f42-83dc-1ba460247da4,1dcde910-fb22-5b04-a19c-00a7e4854271,fa3533e9-b794-5876-97f1-8bd2b50e60f8,edb35e48-a8c8-59ed-b5e8-9b667aab7bea,1343c607-cf79-5052-9c4e-d7a642dba258,9bdc2142-6443-5a6b-9c94-ed9aee47107e,11c0b1e8-2732-5701-aacf-f43bbb05226c,54b9b91a-2b16-5ecb-9f38-64e6bbcebbe4,48c59f9d-0d0b-50a7-8d13-e0ef20b6c6d8,a86e6e3c-7687-589a-b27a-6b570784cc70
0,0.475022,0.475319,0.475572,0.475214,0.474878,0.475365,0.475668,0.475990,0.475600,0.475093,...,0.474898,0.475577,0.475592,0.475223,0.475140,0.475644,0.475041,0.475212,0.474913,0.475748
1,0.476867,0.476690,0.476709,0.477285,0.477153,0.477095,0.477588,0.477422,0.477015,0.477123,...,0.476670,0.476010,0.476760,0.476918,0.477223,0.476681,0.476575,0.476887,0.476516,0.476797
2,0.478490,0.477718,0.477897,0.477711,0.478298,0.477590,0.477709,0.478177,0.478462,0.477982,...,0.477694,0.478184,0.478225,0.478232,0.478234,0.478421,0.477671,0.478078,0.477741,0.477478
3,0.475525,0.476283,0.476349,0.477041,0.476387,0.477615,0.478313,0.477785,0.476908,0.477270,...,0.477931,0.476600,0.477012,0.475814,0.477455,0.476641,0.477961,0.476720,0.476559,0.476393
4,0.477340,0.477903,0.477628,0.477585,0.477923,0.477904,0.478458,0.478143,0.478311,0.478241,...,0.478566,0.478140,0.478816,0.477326,0.477963,0.478091,0.478862,0.478046,0.477887,0.477671
5,0.479750,0.480011,0.479239,0.479286,0.479500,0.479396,0.479838,0.479775,0.479876,0.479283,...,0.479611,0.479642,0.480323,0.479822,0.479417,0.480242,0.479526,0.479673,0.479086,0.479346
6,0.477421,0.477357,0.478240,0.477669,0.477329,0.477691,0.477078,0.477280,0.477973,0.477932,...,0.477028,0.478259,0.477568,0.477418,0.477863,0.477988,0.477184,0.478163,0.477667,0.477893
7,0.484115,0.484071,0.484229,0.484425,0.484720,0.484597,0.484079,0.484006,0.484113,0.484703,...,0.484530,0.484019,0.484421,0.484234,0.485046,0.484778,0.484371,0.484210,0.484435,0.484966
8,0.477096,0.477744,0.478446,0.478714,0.478732,0.479881,0.477921,0.478303,0.478153,0.478654,...,0.478548,0.477746,0.477596,0.477787,0.478124,0.478576,0.479075,0.479050,0.478200,0.478048
9,0.476908,0.477082,0.477594,0.476955,0.477249,0.477133,0.477360,0.477471,0.477698,0.477185,...,0.477032,0.478204,0.477078,0.477114,0.477120,0.477747,0.477881,0.477400,0.477143,0.477596


In [ ]:
# recommand part, need to save the recommanded news id

In [22]:
def get_list_id():
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
        "api_key": mail_api_key,
        "server": mail_server
        })

        response = client.lists.get_all_lists()
    except ApiClientError as error:
        print("Error: {}".format(error.text))
        return None
    return response

def get_active_user():
    list_info = get_list_id()
    if list_info != None:
        list_id = list_info['lists'][0]['id']
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
        "api_key": mail_api_key,
        "server": mail_server
        })

        response = client.lists.get_list_members_info(list_id)
    except ApiClientError as error:
        print("Error: {}".format(error.text))
        return None
    user_list = []
    for i in response['members']:
        user_list.append(i['email_address'])
    return user_list

In [ ]:
# get acctive-user-list, and get rec-user matrix

In [23]:
def get_rec_user_matrix(active_user_list, User_matrix):
    rec_user_matrix = pd.DataFrame()
    for user in active_user_list:
        if user in User_matrix.index:
            init_list = User_matrix.loc[user,:]
        else:
            init_list = User_matrix.mean(axis=0)
        init_pd = pd.DataFrame(init_list, columns=[user], index=User_matrix.columns)
        user_vec = init_pd.T
        rec_user_matrix = pd.concat([rec_user_matrix, user_vec])
    return rec_user_matrix

In [24]:
active_user_list = get_active_user()  # get all active user from Mailchimp portal
rec_user_matrix = get_rec_user_matrix(active_user_list, User_matrix)

In [25]:
rec_user_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ruofan.chen19@gmail.com,0.216967,0.202376,0.205783,0.210671,0.197541,0.192533,0.190339,0.190395,0.194611,0.193697,0.197438,0.216526,0.209742,0.200679,0.187961,0.183044,0.200375,0.19614,0.213883,0.197026
wsc961201@gmail.com,0.216967,0.202376,0.205783,0.210671,0.197541,0.192533,0.190339,0.190395,0.194611,0.193697,0.197438,0.216526,0.209742,0.200679,0.187961,0.183044,0.200375,0.19614,0.213883,0.197026
jun.jin.crf@gmail.com,0.216967,0.202376,0.205783,0.210671,0.197541,0.192533,0.190339,0.190395,0.194611,0.193697,0.197438,0.216526,0.209742,0.200679,0.187961,0.183044,0.200375,0.19614,0.213883,0.197026
ruofan_chen@163.com,0.216967,0.202376,0.205783,0.210671,0.197541,0.192533,0.190339,0.190395,0.194611,0.193697,0.197438,0.216526,0.209742,0.200679,0.187961,0.183044,0.200375,0.19614,0.213883,0.197026
ruofan.chen@uconn.edu,0.216967,0.202376,0.205783,0.210671,0.197541,0.192533,0.190339,0.190395,0.194611,0.193697,0.197438,0.216526,0.209742,0.200679,0.187961,0.183044,0.200375,0.19614,0.213883,0.197026


In [26]:
User_matrix_new = pd.concat([User_matrix, rec_user_matrix], axis=0)
User_matrix_new.to_csv('dataset/P_Q_matrix/User_matrix_u1.csv')

In [27]:
# function
def user_rec_first_day(predict_matrix):  # it's the first day recommendation, just run for initial
    user_news = {}
    for i in range(predict_matrix.shape[0]):
        user_news[predict_matrix.index[i]] = list(predict_matrix.iloc[[i]].sort_values(by=predict_matrix.index[i], ascending=False, axis=1).iloc[:,:10].columns)
    return user_news

In [ ]:
# get predict_matrix

In [28]:
predict_matrix = rec_user_matrix.dot(rec_news_matrix)

In [ ]:
# get recommend news

In [30]:
user_news = user_rec_first_day(predict_matrix)
user_news

{'ruofan.chen19@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a'],
 'wsc961201@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a'],
 'jun.jin.crf@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',

In [31]:
save_dic_to_json(user_news, 'dataset/record/record.json')

True

In [ ]:
# form page and send email

In [32]:
def read_template(filename):
    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)

def save_mail_info(rec_raw_dict, user_news):
    dataset = {}
    for i in range(len(rec_raw_dict['data'])):
        new_key = rec_raw_dict['data'][i]['id']
        dataset[new_key]=rec_raw_dict['data'][i]
    for key in dataset:
        dataset[key]['url'] = 'https://newsboard-r.s3.amazonaws.com/news_gen/' + key +'.html'
    email = list(user_news.keys())
    df = pd.DataFrame()
    for i in range(len(email)):
        df = df.append({'email' : email[i], 
        'news1 title' : dataset[user_news[email[i]][0]]['title'],'news1 abstract':dataset[user_news[email[i]][0]]['abstract'],'news1 url':dataset[user_news[email[i]][0]]['url'],
        'news2 title' : dataset[user_news[email[i]][1]]['title'],'news2 abstract':dataset[user_news[email[i]][1]]['abstract'],'news2 url':dataset[user_news[email[i]][1]]['url'],
        'news3 title' : dataset[user_news[email[i]][2]]['title'],'news3 abstract':dataset[user_news[email[i]][2]]['abstract'],'news3 url':dataset[user_news[email[i]][2]]['url'],
        'news4 title' : dataset[user_news[email[i]][3]]['title'],'news4 abstract':dataset[user_news[email[i]][3]]['abstract'],'news4 url':dataset[user_news[email[i]][3]]['url'],
        'news5 title' : dataset[user_news[email[i]][4]]['title'],'news5 abstract':dataset[user_news[email[i]][4]]['abstract'],'news5 url':dataset[user_news[email[i]][4]]['url'],
        'news6 title' : dataset[user_news[email[i]][5]]['title'],'news6 abstract':dataset[user_news[email[i]][5]]['abstract'],'news6 url':dataset[user_news[email[i]][5]]['url'],
        'news7 title' : dataset[user_news[email[i]][6]]['title'],'news7 abstract':dataset[user_news[email[i]][6]]['abstract'],'news7 url':dataset[user_news[email[i]][6]]['url'],
        'news8 title' : dataset[user_news[email[i]][7]]['title'],'news8 abstract':dataset[user_news[email[i]][7]]['abstract'],'news8 url':dataset[user_news[email[i]][7]]['url'],
        'news9 title' : dataset[user_news[email[i]][8]]['title'],'news9 abstract':dataset[user_news[email[i]][8]]['abstract'],'news9 url':dataset[user_news[email[i]][8]]['url'],
        'news10 title' : dataset[user_news[email[i]][9]]['title'],'news10 abstract':dataset[user_news[email[i]][9]]['abstract'],'news10 url':dataset[user_news[email[i]][9]]['url']
         },ignore_index = True)
    df.to_csv('dataset/mail/user_news.csv',index=False)
    return True

def send_email():
    message_template = read_template('dataset/mail/template.txt')
    sender = "newsboardnyt@163.com"
    password = "******************"
    
    # set up the SMTP server
    smtp = smtplib.SMTP_SSL("smtp.163.com", 994)
    smtp.login(sender, password)
    
    with open('dataset/mail/user_news.csv', 'rb') as csv_file:
        csv_reader = csv.reader(codecs.iterdecode(csv_file, 'utf-8'),delimiter = ',')
        # the below statement will skip the first row
        next(csv_reader)
        for lines in csv_reader:
            msg = MIMEMultipart() # create a message
            # add in the actual person name to the message template
            message = message_template.substitute(news1_title=lines[1],news1_abstract=lines[2],news1_url=lines[3],news2_title=lines[4],news2_abstract=lines[5],news2_url=lines[6],
                                                  news3_title=lines[7],news3_abstract=lines[8],news3_url=lines[9],news4_title=lines[10],news4_abstract=lines[11],news4_url=lines[12],
                                                  news5_title=lines[13],news5_abstract=lines[14],news5_url=lines[15],news6_title=lines[16],news6_abstract=lines[17],news6_url=lines[18],
                                                  news7_title=lines[19],news7_abstract=lines[20],news7_url=lines[21],news8_title=lines[22],news8_abstract=lines[23],news8_url=lines[24],
                                                  news9_title=lines[25],news9_abstract=lines[26],news9_url=lines[27],news10_title=lines[28],news10_abstract=lines[29],news10_url=lines[30])

# setup the parameters of the message
            msg['From']=sender
            msg['To']=lines[0]
            msg['Subject']='newsletter'
# add in the message body
            msg.attach(MIMEText(message, 'plain'))
# send the message via the server set up earlier.
            smtp.send_message(msg)
            del msg
        
        

# Terminate the SMTP session and close the connection
    smtp.quit()
    return 'success'

In [33]:
f = open('dataset/rec_news/raw_news/{}.json'.format(today.strftime("%Y-%m-%d")), 'r', encoding='utf-8')
rec_raw_dict = json.load(f)

In [34]:
save_mail_info(rec_raw_dict, user_news)
send_email()

C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be r

'success'

# End of Day 1

In [ ]:
# base on reaction update Utility matrix

In [35]:
# get feedback infomation
feedback = pd.read_csv('dataset/feedback/survey_export_Apr_26_2022.csv')
click = feedback.to_dict('records')
refresh_click = {}
for key in click:
    refresh_click[key['Question 1: your email address:']] = key['Question 2: For the ten news received today, please tick all the news that you have clicked on or are interested in:']
for key in refresh_click:
    if refresh_click[key] != refresh_click[key]: continue
    refresh_click[key] = refresh_click[key].split(',')
    refresh_click[key] = [int(i.split(' ')[1])-1 for i in refresh_click[key]]
for key in refresh_click:
    if refresh_click[key] != refresh_click[key]: continue
    refresh_click[key] = [user_news[key][i] for i in refresh_click[key]]

In [36]:
refresh_click

{'ruofan.chen19@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304'],
 'ruofan.chen@uconn.edu': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d']}

In [37]:
# get recommended news id list
def get_rec_id_list(user_news):
    rec_id_list = []
    for key in user_news.keys():
        rec_id_list.extend(user_news[key])
    rec_id_list = list(set(rec_id_list))
    return rec_id_list

In [38]:
rec_id_list = get_rec_id_list(user_news)
rec_id_list

['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
 '25dce2f7-6dcb-50fd-b753-04831be85753',
 'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
 '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
 '8c1811fd-b728-5cb7-be1e-67155dd6682c',
 'f30a18e7-97c2-55e3-88d9-33ec6172686d',
 '6b8e1461-e1a7-536c-b825-3410da054019',
 '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a',
 '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
 'b360a596-04ea-5099-bdc3-0a2306b37304']

In [ ]:
# load item matrix and user matrix to get predict matrix

In [39]:
# initial added rows and columns
for key in user_news:
    if key not in Util_matrix.index:
        Util_matrix.loc[key] = 0 * np.ones(Util_matrix.shape[1])  # add row: new user
for i in range(len(rec_id_list)):
    Util_matrix[rec_id_list[i]] = np.full(Util_matrix.shape[0], 0.0) # add column: rec news

In [41]:
refresh_click

{'ruofan.chen19@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304'],
 'ruofan.chen@uconn.edu': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d']}

In [81]:
# # update utility matrix
# for key in refresh_click:
#     if refresh_click[key] == refresh_click[key]:
#         for i in range(len(refresh_click[key])):
#             Util_matrix.loc[key][refresh_click[key][i]] = 2
#         user_dislike = list(set(user_news[key]) - set(refresh_click[key]))
#         for j in range(len(user_dislike)):
#             Util_matrix.loc[key][user_dislike[j]] = 1
             
#     else:
#         for i in range(len(user_news[key])):
#             Util_matrix.loc[key][user_news[key][i]] = 0

In [42]:
# update utility matrix
for key in user_news:
    if key in refresh_click:
        for i in range(len(refresh_click[key])):
            Util_matrix.loc[key][refresh_click[key][i]] = 2
        user_dislike = list(set(user_news[key]) - set(refresh_click[key]))
        for j in range(len(user_dislike)):
            Util_matrix.loc[key][user_dislike[j]] = 1
        user_unseen = list(set(rec_id_list) - set(user_news[key]))
        for j in range(len(user_unseen)):
            Util_matrix.loc[key][user_unseen[j]] = 0
    else:
        user_dislike = user_news[key]
        for j in range(len(user_dislike)):
            Util_matrix.loc[key][user_dislike[j]] = 1
        user_unseen = list(set(rec_id_list) - set(user_news[key]))
        for j in range(len(user_unseen)):
            Util_matrix.loc[key][user_unseen[j]] = 0

In [43]:
Util_matrix

,N8668,N39081,N65259,N79529,N73408,N43615,N29379,N32031,N110232,N101921,...,3cc10c56-ca9d-5dce-8c86-08d003cdaccd,25dce2f7-6dcb-50fd-b753-04831be85753,a4b09ada-bb94-51fc-85ca-0e03f32dda2c,0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9,8c1811fd-b728-5cb7-be1e-67155dd6682c,f30a18e7-97c2-55e3-88d9-33ec6172686d,6b8e1461-e1a7-536c-b825-3410da054019,8d69e5f6-7a3c-5a72-8dd9-41705c961b6a,9bdc2142-6443-5a6b-9c94-ed9aee47107e,b360a596-04ea-5099-bdc3-0a2306b37304
U87243,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U598644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U532401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U593596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U239687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ruofan.chen19@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0
wsc961201@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
jun.jin.crf@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ruofan_chen@163.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [44]:
# save updated Utility matrix
Util_matrix.to_csv('dataset/P_Q_matrix/Util_matrix_d1.csv', index=True)

In [45]:
# since base news changed, update base title dictionary
for i in rec_id_list:
    base_title_dict[i] = rec_title_dict[i]
save_dic_to_json(base_title_dict, 'dataset/base_news/title_news/news_id_title_d1.json')

True

In [46]:
len(base_title_dict.keys())

11927

# For Day 2 

In [47]:
# load Utility Matrix and decompose
Util_matrix = pd.read_csv('dataset/P_Q_matrix/Util_matrix_d1.csv',index_col=0)
utilMat, users_index, items_index = Util_matrix.to_numpy(), list(Util_matrix.index), list(Util_matrix.columns)

User_matrix, Item_matrix = latent_process(utilMat)

# save matrix
Item_matrix = pd.DataFrame(Item_matrix, columns = items_index)
Item_matrix.to_csv('dataset/P_Q_matrix/Item_matrix_d1.csv', index=False)

User_matrix = pd.DataFrame(User_matrix, index=users_index)
User_matrix.to_csv('dataset/P_Q_matrix/User_matrix_d1.csv')

(1005, 11927)
rmse0.497655126515151


In [ ]:
# crawl new recommend news

In [ ]:
# crawl today's news
today = date.today()
year = int(today.strftime("%Y"))
month = int(today.strftime("%m")) # year and month can be modified
day = int(today.strftime("%d"))
raw_news_json = get_initial_news(year, month)

# get text, if failed, add to error list, if failed too much, error list can use to re-get text
error_list = get_dic_text(raw_news_json)

# delete news without text
raw_news_json_copy = copy.deepcopy(raw_news_json)
news_json = clean_empty_text(raw_news_json_copy)
rec_title_list = {}
for news in raw_news_json['data']:
    n_id = news['id']
    title = news['title']
    rec_title_list[n_id] = title
    
# save data
save_dic_to_json(error_list, 'dataset/rec_news/error_list/{}.json'.format(today.strftime("%Y-%m-%d")))
save_dic_to_json(news_json, 'dataset/rec_news/raw_news/{}.json'.format(today.strftime("%Y-%m-%d")))
save_dic_to_json(rec_title_list, 'dataset/rec_news/title_news/{}.json'.format(today.strftime("%Y-%m-%d")))

In [48]:
today = date(2022, 2, 2)

In [49]:
yesday = today - timedelta(days=1)

In [61]:
# use w2v transfer title

In [62]:
# get recommand record and have recommended news id list

In [50]:
f = open('dataset/record/record.json')
record = json.load(f)
rec_id_list = get_rec_id_list(record)
rec_id_list

['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
 '25dce2f7-6dcb-50fd-b753-04831be85753',
 'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
 '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
 '8c1811fd-b728-5cb7-be1e-67155dd6682c',
 'f30a18e7-97c2-55e3-88d9-33ec6172686d',
 '6b8e1461-e1a7-536c-b825-3410da054019',
 '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a',
 '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
 'b360a596-04ea-5099-bdc3-0a2306b37304']

In [51]:
# load today and yesterday news title, then delete have recommended news

In [52]:
f = open('dataset/rec_news/title_news/{}.json'.format(today.strftime("%Y-%m-%d")), 'r', encoding='utf-8')
tod_title_dict = json.load(f)

f = open('dataset/rec_news/title_news/{}.json'.format(yesday.strftime("%Y-%m-%d")), 'r', encoding='utf-8')
yes_title_dict = json.load(f)
len(tod_title_dict.keys()), len(yes_title_dict.keys())

(130, 125)

In [53]:
rec_title_dict = {}
rec_title_dict.update(yes_title_dict)
rec_title_dict.update(tod_title_dict)
len(rec_title_dict.keys())

255

In [54]:
for i in rec_id_list:
    if i in rec_title_dict.keys():
        rec_title_dict.pop(i)
len(rec_title_dict.keys())

245

In [55]:
# load base title dict and cal vec

In [56]:
f = open('dataset/base_news/title_news/news_id_title_d1.json', 'r', encoding='utf-8')
base_title_dict = json.load(f)

In [57]:
# transfer base news {id:title} to {id:vector}
base_title_vec = processing_corpus(base_title_dict, wv)

# transfer recommend news {id:title} to {id:vector}
rec_title_vec = processing_corpus(rec_title_dict, wv)

In [58]:
# cal similarity and transfer recommand news into latent matrix

In [59]:
base_news_pd = pd.DataFrame.from_dict(base_title_vec, orient='index')
rec_news_pd = pd.DataFrame.from_dict(rec_title_vec, orient='index')
rec_news_matrix = cal_rec_matrix(base_news_pd, rec_news_pd, Item_matrix) # this matrix: row:latent (0-19) column: news id

In [60]:
# get have recommended news latent matrix

In [61]:
reced_news_matrix = Item_matrix.loc[:, rec_id_list]

In [62]:
# concate two matrix to form recommend item matrix

In [63]:
rec_news_matrix = pd.concat([reced_news_matrix, rec_news_matrix], axis=1)
rec_news_matrix

,3cc10c56-ca9d-5dce-8c86-08d003cdaccd,25dce2f7-6dcb-50fd-b753-04831be85753,a4b09ada-bb94-51fc-85ca-0e03f32dda2c,0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9,8c1811fd-b728-5cb7-be1e-67155dd6682c,f30a18e7-97c2-55e3-88d9-33ec6172686d,6b8e1461-e1a7-536c-b825-3410da054019,8d69e5f6-7a3c-5a72-8dd9-41705c961b6a,9bdc2142-6443-5a6b-9c94-ed9aee47107e,b360a596-04ea-5099-bdc3-0a2306b37304,...,119b5d31-1516-54b2-aeb3-6e49311099af,34ef43e7-e79d-5ed0-b2fa-648beafaee26,edd37833-255d-5d1f-bc62-4a79953114bb,23a59953-8154-5f72-8dec-5d40f4ae6463,df1d0f85-158f-578d-88b1-6d961284ded7,8fb8321c-a2de-597b-8216-059f0c715654,0afeeab7-2c20-532d-be13-fc57ddb1fd72,7995bb9d-4117-56a0-aef3-e28515b39faf,e8ac3178-085c-56b0-bce2-2d68a82f3f5b,4b0f414a-aee7-5db6-9868-55e1804226bf
0,0.060596,0.262474,0.199430,0.661489,0.178687,0.548452,0.121034,0.117509,0.726605,0.599629,...,0.476447,0.475603,0.476780,0.475703,0.476398,0.476158,0.476195,0.476720,0.476005,0.476528
1,0.138918,0.473799,0.140731,0.137727,0.190947,0.708798,0.845980,0.699590,0.511405,-0.032242,...,0.481264,0.482942,0.483474,0.483301,0.481826,0.482274,0.482968,0.482338,0.483695,0.481585
2,0.461334,0.347555,0.414946,0.156837,0.420353,0.762552,0.060098,0.656399,0.785644,0.275882,...,0.474336,0.475510,0.475063,0.475902,0.474557,0.475286,0.474799,0.475094,0.474830,0.474649
3,0.645068,0.619783,0.730829,0.918697,0.096501,0.417637,0.859863,0.803814,0.545215,0.140205,...,0.479838,0.480657,0.481376,0.480841,0.480178,0.479455,0.480029,0.480291,0.480454,0.480057
4,0.372309,0.432296,0.527456,0.899103,0.598081,0.740304,0.108498,0.418270,0.776524,0.404952,...,0.480972,0.480681,0.480185,0.481074,0.480821,0.480334,0.480325,0.480821,0.480252,0.480234
5,0.485142,-0.066590,0.574768,0.682669,0.062328,0.582629,0.191569,0.541116,0.646230,0.283882,...,0.481582,0.482735,0.482745,0.482907,0.482406,0.482877,0.482835,0.482446,0.483141,0.483054
6,0.916661,0.643490,0.229431,-0.019846,0.864381,0.531193,0.770303,0.635985,0.201615,0.707883,...,0.476061,0.477013,0.477268,0.477432,0.477203,0.476372,0.477682,0.477093,0.477731,0.475504
7,0.054198,0.769906,0.259252,0.412355,0.210717,0.424868,0.033475,0.109420,-0.209340,-0.032357,...,0.478236,0.479268,0.479109,0.479209,0.478630,0.479063,0.479190,0.478619,0.479377,0.478536
8,0.144213,0.952414,0.697419,0.787490,0.262876,0.857694,0.347439,0.780553,0.195409,0.513152,...,0.483388,0.485357,0.483598,0.484359,0.484054,0.482677,0.482601,0.483932,0.483267,0.483212
9,-0.079804,-0.073283,0.630087,0.011118,0.049374,0.763351,0.656288,0.507208,0.312847,0.685701,...,0.475352,0.476395,0.476705,0.476056,0.475463,0.475434,0.475848,0.475339,0.476291,0.474748


In [64]:
Item_matrix_new = pd.concat([Item_matrix, cal_rec_matrix(base_news_pd, rec_news_pd, Item_matrix)], axis=1)
Item_matrix_new.to_csv('dataset/P_Q_matrix/Item_matrix_u2.csv')

In [65]:
Item_matrix_new.shape

(20, 12172)

In [66]:
# get active-user list and active-user matrix

In [67]:
active_user_list = get_active_user()  # get all active user from Mailchimp portal

rec_user_matrix = get_rec_user_matrix(active_user_list, User_matrix)
rec_user_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ruofan.chen19@gmail.com,0.491033,0.424366,0.579045,0.304838,-0.027104,0.534593,0.104338,0.642137,-0.002102,-0.045363,0.252814,0.149608,0.365325,0.658442,0.052936,-0.158899,-0.121366,-0.244218,0.618409,0.582798
wsc961201@gmail.com,-0.235423,0.401957,0.488786,-0.134831,-0.160219,0.326229,0.000482,0.774382,-0.359888,0.241938,0.499141,-0.010743,0.318288,0.082671,0.703711,0.683049,0.591015,-0.296303,0.510291,0.080747
jun.jin.crf@gmail.com,-0.064791,-0.130271,0.325954,-0.014602,0.544050,0.226245,-0.350680,0.608659,-0.397905,0.435508,0.319464,0.145529,0.163092,0.292594,0.262467,0.541935,0.161148,0.228497,0.497760,0.410916
ruofan_chen@163.com,-0.119302,0.041770,-0.085555,0.172082,-0.083576,0.424995,0.458442,0.373510,0.363665,0.652735,0.117289,-0.223884,-0.226107,0.388212,0.369415,0.178007,0.263001,-0.135496,0.430597,0.531399
ruofan.chen@uconn.edu,0.445872,0.424637,-0.132653,-0.245882,-0.036045,0.453308,0.233221,0.015091,0.452001,0.385984,0.334577,0.132082,0.393968,0.270341,0.695495,0.781368,0.142424,0.129870,0.040136,0.123029


In [68]:
old_user_list = list(User_matrix.index) # user already in User matrix
new_user_list = list(set(active_user_list) - set(old_user_list))

In [69]:
User_matrix_new = pd.concat([User_matrix, rec_user_matrix.loc[new_user_list]], axis=0)
User_matrix_new.to_csv('dataset/P_Q_matrix/User_matrix_u2.csv')

In [ ]:
# get predict matrix

In [70]:
predict_matrix = rec_user_matrix.dot(rec_news_matrix)

In [ ]:
# recommend dont push have recommended news

In [71]:
def user_rec(predict_matrix, record):
    user_list = list(predict_matrix.index)
    user_news = {}
    for user in user_list:
        rec_list = []
        score_list = predict_matrix.loc[user]
        sort_score_list = np.sort(score_list)
        id_list = score_list.index
        if user in record.keys():
            reced_news = record[user]
            i = 1
            while len(rec_list) < 10:
                max_score = sort_score_list[-i]
                index = np.where(score_list == max_score)[0][0]
                news_id = id_list[index]
                if news_id not in reced_news:
                    rec_list.append(news_id)
                i += 1
        else:
            for score in sort_score_list[-10:]:
                index = np.where(score_list == score)[0][0]
                news_id = id_list[index]
                rec_list.append(news_id)
        user_news[user] = rec_list
    return user_news

In [72]:
user_news = user_rec(predict_matrix, record)
user_news

{'ruofan.chen19@gmail.com': ['ad9455b8-0333-59cc-9de0-44c2eaf5070e',
  '18cbb1c3-60e8-577a-bc68-61a0f3cc5f35',
  '61feb25e-a176-5043-887f-26e9b97afff7',
  'e85f061b-e82e-564e-946b-7ffa1de77819',
  '42d691a5-c58d-5065-b9ab-e3537866a6d8',
  '6b682e0e-3493-5f9b-86da-3297817c9a2b',
  '12c3b0a4-2026-58d2-b96b-4a9c102f2bab',
  '75dd6213-2bb9-5988-83ac-01ad04d2bb69',
  '1dcde910-fb22-5b04-a19c-00a7e4854271',
  'edd37833-255d-5d1f-bc62-4a79953114bb'],
 'wsc961201@gmail.com': ['ad9455b8-0333-59cc-9de0-44c2eaf5070e',
  '12c3b0a4-2026-58d2-b96b-4a9c102f2bab',
  '23a59953-8154-5f72-8dec-5d40f4ae6463',
  '1294616e-08a1-5407-8d80-3890c04dfa28',
  '75dd6213-2bb9-5988-83ac-01ad04d2bb69',
  '42d691a5-c58d-5065-b9ab-e3537866a6d8',
  '11c0b1e8-2732-5701-aacf-f43bbb05226c',
  'e85f061b-e82e-564e-946b-7ffa1de77819',
  '01e0ffd6-1bbe-5b53-a9b8-4dde2a6b142d',
  '61feb25e-a176-5043-887f-26e9b97afff7'],
 'jun.jin.crf@gmail.com': ['ad9455b8-0333-59cc-9de0-44c2eaf5070e',
  'cefcda4b-2eb7-5a12-a8ee-177d55127974',

In [ ]:
# form page and send email

In [73]:
f = open('dataset/rec_news/raw_news/{}.json'.format(today.strftime("%Y-%m-%d")), 'r', encoding='utf-8')
tod_raw_dict = json.load(f)

In [74]:
f = open('dataset/rec_news/raw_news/{}.json'.format(yesday.strftime("%Y-%m-%d")), 'r', encoding='utf-8')
yes_raw_dict = json.load(f)

In [ ]:
# combine 2 days raw news in to one dict

In [75]:
rec_raw_dict = {}
tod_raw_list = tod_raw_dict['data']
yes_raw_list = yes_raw_dict['data']
rec_raw_list = []
rec_raw_list.extend(yes_raw_list)
rec_raw_list.extend(tod_raw_list)
rec_raw_dict['data'] = rec_raw_list

In [76]:
save_mail_info(rec_raw_dict, user_news)
send_email()

C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'email' : email[i],
C:\cygwin64\tmp\ipykernel_17152\1373300779.py:16: FutureWarning: The frame.append method is deprecated and will be r

'success'

In [ ]:
# add today recommend news into record

In [77]:
record

{'ruofan.chen19@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a'],
 'wsc961201@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',
  '25dce2f7-6dcb-50fd-b753-04831be85753',
  '6b8e1461-e1a7-536c-b825-3410da054019',
  '8c1811fd-b728-5cb7-be1e-67155dd6682c',
  'f30a18e7-97c2-55e3-88d9-33ec6172686d',
  '9bdc2142-6443-5a6b-9c94-ed9aee47107e',
  '0f0d8266-d4d6-54b1-81b2-e0b16d3e71e9',
  'b360a596-04ea-5099-bdc3-0a2306b37304',
  '8d69e5f6-7a3c-5a72-8dd9-41705c961b6a'],
 'jun.jin.crf@gmail.com': ['3cc10c56-ca9d-5dce-8c86-08d003cdaccd',
  'a4b09ada-bb94-51fc-85ca-0e03f32dda2c',

In [78]:
def combine_record(user_news, record):
    new_record = {}
    for i in list(user_news.keys()):
        total_rec_list = []
        tod_rec_list = user_news[i]
        if i in record.keys():
            record_rec_list = record[i]
            total_rec_list.extend(tod_rec_list)
            total_rec_list.extend(record_rec_list)
        else:
            total_rec_list = tod_rec_list
        new_record[i] = total_rec_list
    return new_record

In [79]:
record_new = combine_record(user_news, record)

In [80]:
save_dic_to_json(record_new, 'dataset/record/record.json') # cover record file

True

# End of Day 2 : same as End of Day 1

# iterate everyday